In [1]:
import pandas as pd
import numpy as np
import time
# LDA, tSNE
from sklearn.manifold import TSNE
from gensim.models.ldamodel import LdaModel
# NLTK
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re
# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline
import seaborn as sns
# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider
from bokeh.layouts import column
from bokeh.palettes import all_palettes
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv('Tweets.csv')

In [3]:
df.head()

Unnamed: 0                                             tweets  likes  \
0           0  @SujataIndia1st @Trendsetting123 @narendramodi...      0   
1           1  Construction now been stopped with immediate e...      0   
2           2  Respected @narendramodi Ji 🙏\nIt's a responsib...      0   
3           3  @drharshvardhan @WHO @JaneEllison @DrTedros @P...      0   
4           4  @drharshvardhan @PMOIndia @narendramodi @BJP4I...      0   

                  time  
0  2021-05-13 14:48:38  
1  2021-05-13 14:48:35  
2  2021-05-13 14:48:34  
3  2021-05-13 14:48:26  
4  2021-05-13 14:48:24

In [4]:
df['time'] = df['time'].astype('datetime64[ns]')
df['time'] = df['time'].astype(np.int64) / int(1e6)

In [5]:
print(df.tweets[0][:500])

@SujataIndia1st @Trendsetting123 @narendramodi @RahulGandhi @MaryDenniz Conniving somewhere how to implement more illegal activities 😣 where ekes


**Initial cleaning**

In [6]:
%%time
# Removing numerals:
df['tweet_tokens'] = df.tweets.map(lambda x: re.sub(r'\d+', '', x))
# Lower case:
df['tweet_tokens'] = df.tweet_tokens.map(lambda x: x.lower())
print(df['tweet_tokens'][0][:500])

@sujataindiast @trendsetting @narendramodi @rahulgandhi @marydenniz conniving somewhere how to implement more illegal activities 😣 where ekes
Wall time: 15.6 ms


**Tokenize**

In [7]:
%%time
df['tweet_tokens'] = df.tweet_tokens.map(lambda x: RegexpTokenizer(r'\w+').tokenize(x))
print(df['tweet_tokens'][0][:25])

['sujataindiast', 'trendsetting', 'narendramodi', 'rahulgandhi', 'marydenniz', 'conniving', 'somewhere', 'how', 'to', 'implement', 'more', 'illegal', 'activities', 'where', 'ekes']
Wall time: 0 ns


**Stemming**

In [8]:
%%time
snowball = SnowballStemmer("english")  
df['tweet_tokens'] = df.tweet_tokens.map(lambda x: [snowball.stem(token) for token in x])
print(df['tweet_tokens'][0][:25])

['sujataindiast', 'trendset', 'narendramodi', 'rahulgandhi', 'marydenniz', 'conniv', 'somewher', 'how', 'to', 'implement', 'more', 'illeg', 'activ', 'where', 'eke']
Wall time: 0 ns


**Stop Words**

In [9]:
%%time
stop_en = stopwords.words('english')
df['tweet_tokens'] = df.tweet_tokens.map(lambda x: [t for t in x if t not in stop_en]) 
print(df['tweet_tokens'][0][:25])

['sujataindiast', 'trendset', 'narendramodi', 'rahulgandhi', 'marydenniz', 'conniv', 'somewher', 'implement', 'illeg', 'activ', 'eke']
Wall time: 62.5 ms


**Final Cleaning**

In [10]:
%%time
df['tweet_tokens'] = df.tweet_tokens.map(lambda x: [t for t in x if len(t) > 1])
print(df['tweet_tokens'][0][:25])

['sujataindiast', 'trendset', 'narendramodi', 'rahulgandhi', 'marydenniz', 'conniv', 'somewher', 'implement', 'illeg', 'activ', 'eke']
Wall time: 0 ns


**LDA**

In [11]:
from gensim import corpora, models
np.random.seed(2017)
texts = df['tweet_tokens'].values
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = models.ldamodel.LdaModel(corpus, id2word=dictionary, 
                                    num_topics=8, passes=5, minimum_probability=0)

In [12]:
ldamodel.print_topics()

[(0,
  '0.038*"mess" + 0.038*"amitshah" + 0.038*"janeellison" + 0.038*"sure" + 0.038*"drtedro" + 0.038*"jpnadda" + 0.038*"mohfw_india" + 0.038*"situat" + 0.038*"bjpindia" + 0.038*"drharshvardhan"'),
 (1,
  '0.010*"narendramodi" + 0.010*"rahulgandhi" + 0.010*"bjpindia" + 0.010*"drharshvardhan" + 0.010*"rubikaliyaquat" + 0.010*"jpnadda" + 0.010*"moayush" + 0.010*"co" + 0.010*"peopl" + 0.010*"eke"'),
 (2,
  '0.091*"idea" + 0.090*"ani" + 0.047*"time" + 0.046*"vaccin" + 0.025*"moayush" + 0.025*"ayushmannha" + 0.025*"amitshah" + 0.025*"product" + 0.025*"peopl" + 0.025*"narendramodi"'),
 (3,
  '0.048*"narendramodi" + 0.047*"somewher" + 0.047*"rahulgandhi" + 0.047*"trendset" + 0.047*"conniv" + 0.047*"activ" + 0.047*"illeg" + 0.047*"implement" + 0.047*"sujataindiast" + 0.047*"marydenniz"'),
 (4,
  '0.052*"narendramodi" + 0.028*"rajatsharmal" + 0.028*"anjanaomkashyap" + 0.028*"saansad" + 0.028*"sudhirchaudhari" + 0.028*"respons" + 0.028*"respect" + 0.028*"rubikaliyaquat" + 0.028*"nynryeafg" + 0.

In [13]:
hm = np.array([[y for (x,y) in ldamodel[corpus[i]]] for i in range(len(corpus))])

In [14]:
#And reduce dimensionality using t-SNE algorithm:
tsne = TSNE(random_state=2017, perplexity=30, early_exaggeration=120)
embedding = tsne.fit_transform(hm)
embedding = pd.DataFrame(embedding, columns=['x','y'])
embedding['hue'] = hm.argmax(axis=1)

**PLOTTING**

In [15]:
source = ColumnDataSource(
        data=dict(
            x = embedding.x,
            y = embedding.y,
            colors = [all_palettes['Set1'][8][i] for i in embedding.hue],
            title = df.tweets,
            day = df.time,
            alpha = [0.9] * embedding.shape[0],
            size = [7] * embedding.shape[0]
        )
    )
hover_tsne = HoverTool(names=["df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Title:</span>
            <span style="font-size: 12px">@title</span>
            <span style="font-size: 12px; font-weight: bold;">Day:</span>
            <span style="font-size: 12px">@time</span>
        </div>
    </div>
    """)
tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']
plot_tsne = figure(plot_width=700, plot_height=700, tools=tools_tsne, title='Papers')
plot_tsne.circle('x', 'y', size='size', fill_color='colors', 
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source, name="df")

callback = CustomJS(args=dict(source=source), code=
    """
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    colors = data['colors']
    alpha = data['alpha']
    title = data['title']
    time = data['time']
    size = data['size']
    for (i = 0; i < x.length; i++) {
        if (time[i] <= f) {
            alpha[i] = 0.9
            size[i] = 7
        } else {
            alpha[i] = 0.05
            size[i] = 4
        }
    }
    source.change.emit();
    """)

slider = Slider(start=df.time.min(), end=df.time.max(), value=2016, step=1, title="Day of tweets")
slider.js_on_change('value', callback)

layout = column(slider, plot_tsne)
show(layout)